# Parsing cloudfront logs

In [ ]:
import pandas as pd
import scipy as sp
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json
from datetime import datetime, timedelta
from urllib import parse
from collections import Counter, defaultdict
matplotlib.rcParams['figure.figsize'] = [16, 9]

Let's start by opening the searches json we created, following the instructions in the readme, and doing a bit of ad-hoc investigation to sense check it. Well look at the number of searches, the most common query terms, and the least common query terms:

In [ ]:
with open("data/searches_nov_23.json") as file:
    searches = json.load(file)

In [ ]:
len(searches)

In [ ]:
query_counter = Counter()
for search in searches:
    if "query" in search["query"] and len(search["documents"]) > 0:
        query_counter.update(search["query"]["query"])
query_counter.most_common(20)

In [ ]:
query_counter.most_common()[-20:-1]

We can also print a wordcloud, removing neutral citations and some of the more common bits of bot spam, to get an overview of the search themes:

In [ ]:
nc_match = '(fam|ewca|civ|ewhc|crim|ewfc|ewca|crown court| v )'
bot_match  = '(xor|sysdate|select|sleep|waitfor|delay|search|pipe|dbms|recieve|message)'
exclude = ["the", "hrc", "1", "1\\]
from wordcloud import WordCloud
queries = []
for search in searches:
    if "query" in search["query"]:
        if not re.search(nc_match, search["query"]["query"][0], re.IGNORECASE):
            if not re.search(bot_match, search["query"]["query"][0], re.IGNORECASE):
                queries += [parse.unquote(q).strip().lower() for q in search["query"]["query"]]
wordcloud = WordCloud(background_color="white", max_words=5000, contour_width=3, contour_color='steelblue', width=1920, height=1080)
wordcloud.generate(" ".join(queries))
wordcloud.to_image()

How many searches resulted in a click on a document?

In [ ]:
searches_with_query_and_documents = [search for search in searches if "query" in search["query"] and len(search["documents"]) > 0]

In [ ]:
len(searches_with_query_and_documents)

We want to clean up these queries a bit, there's still lots of bot spam to be dealt with. To do that, we'll make a CSV of the the query terms that appear in the searches, and the number of searches with that query:

In [ ]:
query_counter = Counter([parse.unquote(s["query"]["query"][0]).strip().lower() for s in searches_with_query_and_documents])
query_counts_df = pd.DataFrame.from_records([{"query": q[0], "count": q[1]} for q in query_counter.most_common()])

In [ ]:
len(query_counter)

In [ ]:
query_counts_df.to_csv("data/query_counts.csv")

We can then open that CSV in our favourite spreadsheet, and add a new column with the heading 'exclude', adding a '1' in every row we want to exclude from our test set. This shouldn't take that long - I spent about 20 minutes on it in a set with ~9000 queries. Then load that CSV in here, and continue.


In [ ]:
exclusions = pd.read_csv("data/query_counts_with_exclusions.csv", sep=";")
exclusions = exclusions[exclusions.exclude == 1][["query", "count"]]

We create a new set of queries and documents excluding the ones we've marked, and check its length, outputting that file to `data/search_queries_and_documents.json` where our search metrics notebook can use it.

In [ ]:
searches_with_query_and_documents_without_exclusions = []
for search in searches_with_query_and_documents:
    query = search["query"]["query"][0]
    if parse.unquote(query).strip().lower() not in set(exclusions["query"]):
        searches_with_query_and_documents_without_exclusions.append(search)

In [ ]:
len(searches_with_query_and_documents_without_exclusions)

In [ ]:
with open("data/search_queries_and_documents.json", "w") as file:
    json.dump(searches_with_query_and_documents_without_exclusions, file, default= lambda x: x.isoformat())

Finally, let's calculate the proportion of abandoned searches:

In [ ]:
searches_with_query = [search for search in searches if "query" in search["query"]]

In [ ]:
searches_with_query_without_exclusions = []
for search in searches_with_query:
    query = search["query"]["query"][0]
    if parse.unquote(query).strip().lower() not in set(exclusions["query"]):
        searches_with_query_without_exclusions.append(search)

In [ ]:
len(searches_with_query_and_documents_without_exclusions) / float(len(searches_with_query_without_exclusions))